In [1]:
import os
import random
import numpy as np
import pandas as pd
import math

from glob import glob
from IPython.display import display

#### Read dataset

In [2]:
#Defining original dataset directory.
Path = 'c://Datasets//Multiagent//rtb'

#Importing data.
os.chdir(Path)
Datasets_features = {}
for Filename in glob('*.csv'):
    Datasets_features[Filename[:-4].title()] = pd.read_csv(Filename, sep = ',')
    
#Defining variables.
Train_ftr = Datasets_features['Train']
Validation_ftr = Datasets_features['Validation']

#### Define general functions

In [3]:

#Linear bidding strategy.
def linear_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*pCTR/Init_CTR)

def nlogn_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*(pCTR/Init_CTR)*np.log(pCTR/Init_CTR))

def exp_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*np.e**(pCTR/Init_CTR))

def ortb_bidding(pCTR, c=50, lmda=5.2e-7, ortbtype=2):
    dim = pCTR.shape[0]
    if ortbtype==2: 
        expr = (np.array(pCTR) + np.sqrt(np.repeat(c, dim) ** 2 * np.repeat(lmda, dim) * 2 + np.repeat(c, dim) ** 2)) / (np.repeat(c, dim) * np.repeat(lmda, dim))
        return = np.repeat(c, dim) * (expr ** (1 / 3) - (1 / expr) ** (1 / 3))  
    else:
        return = np.sqrt(np.repeat(c, dim) / np.repeat(lmda, dim) * np.array(pCTR) + np.repeat(c, dim) ** 2) - np.repeat(c, dim)

#Winning criterion #1.
def win_auction_1(bid, payprice):
    return bid >= payprice
    
#Winning criterion #2.
def win_auction_2(bid, payprice, Others):
    return ((bid >= payprice) and (bid >= max(Others)))
    
#Bidding strategy parameter eveluation.
def Bidding_strategy_prm(Budget, Init_CTR, Init_eCPC, Clicks_Prices, 
                         Pred_CTRs, prm, Dict_alg, key, curr_strat_vals,alg):
    
    empCPM = []
    Bids, Cost, Imps, Clks = 0, 0, 0, 0
    
    #Looping on bid requests.
    for item in range(0, len(Clicks_Prices)):
        bid = 0
        Bids += 1
        pCTR = Pred_CTRs[item]           
            
        #Linear strategy.
        if key == 'Linear':
            Criterion = 1
            bid = linear_bidding(prm, pCTR, Init_CTR)
            Win = win_auction_1(bid, Clicks_Prices[item][1])
        # NLogN strategy 
        elif key == 'NLogN':
            Criterion = 1
            bid = nlogn_bidding(prm, pCTR, Init_CTR)
            Win = win_auction_1(bid, Clicks_Prices[item][1])
        elif key == 'Exp':
            Criterion = 1
            bid = exp_bidding(prm, pCTR, Init_CTR)
            Win = win_auction_1(bid, Clicks_Prices[item][1])
        elif key == 'Ortb':
            Criterion = 1
            bid = ortb_bidding(pCTR, prm[0], prm[1])
            Win = win_auction_1(bid, Clicks_Prices[item][1])                    
        #Exception: error.
        else:
            print('[Strategy error]: Wrong bidding strategy name!')
            break
            
        #Computing bid outcome.
        if Win:
            Imps += 1
            Clks += Clicks_Prices[item][0]
            
            if Criterion == 1:
                Cost += Clicks_Prices[item][1]/1000
                empCPM.append(Clicks_Prices[item][1])
            else:
                Cost += max(Clicks_Prices[item][1],max(others))/1000
                empCPM.append(max(Clicks_Prices[item][1],max(others)))
                
        #Stopping condition.
        if Cost > Budget:
            break
            
    #Reporting results.
    if Imps != 0: empCTR = Clks/Imps
    else: empCTR = 0
    
    avgCPM = np.mean(empCPM)
    if Clks != 0: avgCPC = np.sum(empCPM)/(Clks*1000)
    else: avgCPC = 0
    
    Output =  key + ',' + alg + ',' + str(prm) + ',' + str(Budget) + ',' + str(Cost) + ','\
              + str(Bids) + ',' + str(Imps) + ',' + str(Clks) + ','\
              + str(empCTR) + ',' + str(avgCPM) + ',' + str(avgCPC)
    #temp_df = pd.DataFrame(,columns=['Algorithm', 'Parameter', 'Budget','Spent','Imps', 'Click','eCTR','avgCPM','avgCPC'])
    curr_strat_vals.loc[len(curr_strat_vals)]=[key,alg,str(prm),Budget,Cost,Bids,Imps,Clks,empCTR,avgCPM,avgCPC]
    return Output
    
#Bidding strategy simulation.
def Bidding_strategy(Budget, Init_CTR, Init_eCPC, Clicks_Prices, Pred_CTRs,
                     file, Dict_alg, key, curr_strat_vals, alg):
    parameters = Dict_alg[key]
    for prm in parameters:
        Output = Bidding_strategy_prm(Budget, Init_CTR, Init_eCPC, Clicks_Prices,
                                      Pred_CTRs, prm, Dict_alg, key, curr_strat_vals, alg)
        file.write(Output + '\n')

#Actual bid creation and writing into file, based on baseB
def createBids(bidIds,baseBid,pred_CTRs,init_CTR,alg_name, key ):
    Filename = 'Team_07_'+ alg_name +'_'+ key+ '_bids' +'.csv'
    print('Using best param...' + '{:.3f}'.format(baseBid))
    with open(Filename, 'w') as file:
        file.write('bidid,bidprice' + '\n')
        for idx in range(len(bidIds)):
            bidId = bidIds[idx]
            pCTR  = pred_CTRs[idx]
            bid = 0
            if key == "Linear":
                bid   = linear_bidding(baseBid,pCTR, init_CTR)
            elif key == 'NLogN':
                bid = nlogn_bidding(baseBid, pCTR, init_CTR)
            elif key == 'Exp':
                bid = exp_bidding(baseBid,pCTR,init_CTR)
            else:
                raise NameError('key not found')
            
            #print(bidId,' ', baseBid,' ',pCTR,' ',init_CTR,' ', bid)
            file.write(bidId + ',' + '{:.1f}'.format(bid) +'\n')
            
    print('• [File %s]: Process completed!' %Filename)

#### Read CTR information, base bid optimisation and bid creation
For CTR, two files should be in the folder, with the next format: '<Algorithm\>\_test\_pCTR.csv' and '<Algorithm\>\_validation\_pCTR.csv' (Note: Currently case sensitive). Then, the algorithm name shall be included in the _algs_ array.

In [3]:
import itertools

#Defining CTR directory.
Path = 'C://Datasets//Multiagent//calc//pctrs'

#Importing data.
os.chdir(Path)
Datasets_CTR = {}
overall_alg_vals = pd.DataFrame(columns=['BidStrat','Algorithm', 'Parameter', 'Budget','Spent','Bids','Imps', 'Click','eCTR','avgCPM','avgCPC'])

for Filename in glob('*_pCTR.csv'):
    Datasets_CTR[Filename[:-4]] = pd.read_csv(Filename, sep = ',')

#Define algorithms to use
algs = [ 'NN','LR','Catboost','LGBM']
#',


#Initialising values.
Init_CTR = sum(Train_ftr['click'])/len(Train_ftr)
Init_eCPC = sum(Train_ftr['payprice'])/(sum(Train_ftr['click'])*1000)
print("avgCTR: ",Init_CTR)

PathBB =   'c://Datasets//Multiagent//calc//bb'
PathBids = 'c://Datasets//Multiagent//calc//bids'
    
for alg in algs:
    #Defining variables.
    Test_CTR = Datasets_CTR[alg+'_test_pCTR']
    Validation_CTR = Datasets_CTR[alg+'_validation_pCTR']
    #Training bidding strategies.
    Budget = 6250
    random.seed(10)

    
    
    #Defining general variables.
    Pred_CTRs = []
    Clicks_Prices = []

    #Initialising parameters.
    Prm_lin = np.concatenate((np.arange(2, 20, 2),np.arange(20, 100, 5),
                              np.arange(100, 400, 10), np.arange(400, 800, 50)),
                              axis=0)
    Prm_log = np.logspace(1,4,num=25)
    #Prm_95 = [95]
    
    c_range = np.linspace(1, 80, 40) 
    lmda_range = np.linspace(1e-5, 1, 100)
    
    Prm_ortb = list(itertools.product(c_range,lmda_range))  
    
    Dict_alg = {'Linear': Prm_lin,
               'Ortb': Prm_ortb}#,,  ,'Exp': Prm_lin,'NLogN': Prm_log
    
    Pred_CTRs = list(Validation_CTR['pCTR'].values)
    Pred_CTRs_Test = list(Test_CTR['pCTR'].values)
    Clicks_Prices = list(zip(Validation_ftr['click'], Validation_ftr['payprice']))

    #Training and saving results.
    os.chdir(PathBB)

    Header = 'bidStrat,algorithm,parameter,budget,spend,bids,imps,click,empCTR,avgCPM,avgCPC'

    for key in Dict_alg.keys():
        print('Processing: ', alg,'-',key)
        os.chdir(PathBB)
        Filename = key + '_BB_'+ alg+'.csv'
        curr_alg_vals = pd.DataFrame(columns=['BidStrat','Algorithm', 'Parameter', 'Budget','Spent','Bids', 'Imps','Click','eCTR','avgCPM','avgCPC'])
        with open(Filename, 'w') as file:
            file.write(Header + '\n')
            Bidding_strategy(Budget, Init_CTR, Init_eCPC, Clicks_Prices,
                             Pred_CTRs, file, Dict_alg, key, curr_alg_vals, alg)
        print('• [File %s]: Process completed!' %Filename)
        overall_alg_vals = overall_alg_vals.append(curr_alg_vals.sort_values('Click', ascending=False).head(2), sort=False)
        bestParam = np.single(curr_alg_vals.sort_values('Click', ascending=False).head(1).iloc[0,2])
        os.chdir(PathBids)
        createBids(Test_CTR['bidid'],bestParam,Pred_CTRs_Test,Init_CTR,alg,key)

print('• Full process completed!')
print(overall_alg_vals.sort_values('Click', ascending=False))

[(1.0, 0.1), (1.0, 1.1090909090909091), (1.0, 2.118181818181818), (1.0, 3.1272727272727274), (1.0, 4.136363636363636), (1.0, 5.145454545454545), (1.0, 6.154545454545454), (1.0, 7.163636363636363), (1.0, 8.172727272727272), (1.0, 9.181818181818182), (1.0, 10.19090909090909), (1.0, 11.2), (1.0, 12.209090909090909), (1.0, 13.218181818181817), (1.0, 14.227272727272727), (1.0, 15.236363636363635), (1.0, 16.245454545454546), (1.0, 17.254545454545454), (1.0, 18.263636363636365), (1.0, 19.272727272727273), (1.0, 20.28181818181818), (1.0, 21.290909090909093), (1.0, 22.3), (1.0, 23.30909090909091), (1.0, 24.31818181818182), (1.0, 25.327272727272728), (1.0, 26.336363636363636), (1.0, 27.345454545454544), (1.0, 28.354545454545455), (1.0, 29.363636363636363), (1.0, 30.37272727272727), (1.0, 31.381818181818183), (1.0, 32.39090909090909), (1.0, 33.4), (1.0, 34.40909090909091), (1.0, 35.418181818181814), (1.0, 36.42727272727273), (1.0, 37.43636363636364), (1.0, 38.445454545454545), (1.0, 39.4545454545

'\n#Defining CTR directory.\nPath = \'C://Datasets//Multiagent//calc//pctrs\'\n\n#Importing data.\nos.chdir(Path)\nDatasets_CTR = {}\noverall_alg_vals = pd.DataFrame(columns=[\'BidStrat\',\'Algorithm\', \'Parameter\', \'Budget\',\'Spent\',\'Bids\',\'Imps\', \'Click\',\'eCTR\',\'avgCPM\',\'avgCPC\'])\n\nfor Filename in glob(\'*_pCTR.csv\'):\n    Datasets_CTR[Filename[:-4]] = pd.read_csv(Filename, sep = \',\')\n\n#Define algorithms to use\nalgs = [ \'NN\',\'LR\',\'Catboost\',\'LGBM\']\n#\',\n\n\n#Initialising values.\nInit_CTR = sum(Train_ftr[\'click\'])/len(Train_ftr)\nInit_eCPC = sum(Train_ftr[\'payprice\'])/(sum(Train_ftr[\'click\'])*1000)\nprint("avgCTR: ",Init_CTR)\n\nPathBB =   \'c://Datasets//Multiagent//calc//bb\'\nPathBids = \'c://Datasets//Multiagent//calc//bids\'\n    \nfor alg in algs:\n    #Defining variables.\n    Test_CTR = Datasets_CTR[alg+\'_test_pCTR\']\n    Validation_CTR = Datasets_CTR[alg+\'_validation_pCTR\']\n    #Training bidding strategies.\n    Budget = 6250\n  

In [ ]:
overall_alg_vals.sort_values('Click', ascending=False)

In [ ]:
import winsound
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)